# Regresión Logística con Base de datos de Mercadeo de Bancos

## Importar las librerías

In [1]:
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense

## Ordenar los datos

In [2]:
df = pd.read_csv("bank.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


#### Convertir los 'yes'/'no' en valores numéricos (0 y 1)

In [3]:
df['y'] = [0 if x == 'no' else 1 for x in df['y']] #Reemplazamos los 'yes' 'no' por valores '0' y '1'

X = df.drop('y', 1) #dejamos la columna de caracteres
Y = df.y  #elegimos la columna de los resultados en números

Y.value_counts()

0    4000
1     521
Name: y, dtype: int64

* Hay 4000 'no'   y   521 'yes'

## Se analizan las columnas con categorías y se vuelven numéricas

In [4]:
catego = []
for col_name in X.columns:
    if X[col_name].dtypes == 'object':# in pandas it is object
        catego.append(col_name) #Se añaden las categorías a la lista
        unique_cat = len(X[col_name].unique())

* Para volver una columna categórica en numérica se crean 'variables dummies'
* Estas representan cada una de las categorías y se asignan con '0' ó '1'

In [5]:
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False) # prefix give name
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [6]:
X = dummy_df(X, catego)
catego_dummies = list(X.columns)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

## Se intenta con un modelo de clasificación de SVM

In [8]:
cls = svm.SVC().fit(X_train, Y_train)

In [9]:
preds_train = cls.predict(X_train) # Se predicen los valores los datos de entrenamiento
print("Accuracy:",metrics.accuracy_score(Y_train, preds_train)) #Se comparan con los resultados reales

Accuracy: 0.8864306784660767


In [10]:
preds_test = cls.predict(X_test) # Se predicen los valores los datos de entrenamiento
print("Accuracy:",metrics.accuracy_score(Y_test, preds_test)) #Se comparan con los resultados reales

Accuracy: 0.8822553897180763


# Comparando con Redes Neuronales

In [11]:
X = X.values
Y = Y.values

## Se Regularizan los datos y divide en diferentes sets

In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

In [13]:
# Se dividen los datos en Training, Testing y Validation set
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, Y, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [14]:
print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(3164, 51) (678, 51) (679, 51) (3164,) (678,) (679,)


### Se crea el modelo neuronal y se compila

In [15]:
model = Sequential([
                    Dense(32, activation='relu', input_shape=(51,)), #Capa de entrada (51 datos de entrada o columnas)
                    Dense(32, activation='relu'),                    #Capa escondida 
                    Dense(1, activation='sigmoid'),              #Capa final (como es clase binaria se usa sigmoide y 1 nodo)
                    ])

In [16]:
model.compile(optimizer='sgd', 
              loss='binary_crossentropy',           
              metrics=['accuracy'])

### Se ajusta la red neuronal a los datos

In [17]:
hist = model.fit(X_train, Y_train, 
                 epochs=20,     
                 validation_data=(X_val, Y_val), verbose=False)

In [18]:
evaluacion = model.evaluate(X_test, Y_test, verbose=False)

print("La función de costo es de: ", evaluacion[0])
print("\nLa precisión de la NN es de: ", evaluacion[1])

La función de costo es de:  0.3044069707393646

La precisión de la NN es de:  0.8895434737205505


## Tiene el mismo resultado utilizar un algoritmo de SVM a una NN
### SVM precisión de 0.88
### NN precisión de 0.87